# Compute the neon K-LL Auger spectrum

In [1]:
using JAC

In this tutorial, we wish to calculate the K-LL spectrum of atomic neon after $1s$ inner-shell ionization as it may occurs, for instance, following photon, electron or neutron impact. An autoionization of an atom or ion generally occurs if the initially-excited level is **energetically embedded into the continuum of the next higher charge state**, and then often also dominates the decay of the atom.

Since the neon K-shell ($1s$) electron has a binding energy larger than 800 eV, this condition is fulfilled and may, in priciple, even lead to a 4-5 times multiple ionization, although the initial excitation energy does not tell much of how many electrons will be emitted eventually. This depends of course on the particular decay path
of an atom and how easily further electrons can be *shaked-up* (or *off*) the atom during the de-excitation.


An Auger electron emission often leads to characteristic and energetically isolated lines as the kinetic energy of the emitted electrons are given simply by the difference of the (total) initial- and final-level energies. We therefore need to specify only the initial- and final-state multiplets as well as the process identifier `Auger` in order to calculate the K-LL spectrum with just the standard settings, but by defining a grid that is appropriate to describe *outgoing* electrons:

In [2]:
grid = Radial.Grid(Radial.Grid(true), rnt = 2.0e-6, h = 5.0e-2, hp = 1.0e-2, rbox = 10.)
wa   = Atomic.Computation(Atomic.Computation(), name="Neon K-LL Auger spectrum",  grid=grid,
              nuclearModel=Nuclear.Model(10.),
              initialConfigs=[Configuration("1s 2s^2 2p^6")],
              finalConfigs  =[Configuration("1s^2 2s^2 2p^4"), Configuration("1s^2 2s 2p^5"), 
                              Configuration("1s^2 2p^6")], processSettings=AutoIonization.Settings() )
perform(wa)


... in performSCF ...
>> include Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 10.0 with mass = 20.5, radius R = 2.8580081360657426 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -5.00667217e+01    -5.00667420e+01    +4.05066052e-07    
      2      2s_1/2      -1.25208571e+01    -1.25208597e+01    +2.03418026e-07    
      3      3s_1/2      -5.56296960e+00    -5.56297036e+00    +1.35691297e-07    
      4      4s_1/2      -3.12838775e+00    -

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_3/2      -1.25041630e+01    -1.25041630e+01    -4.87554157e-13    
      2      3p_3/2      -5.55802266e+00    -5.55802266e+00    -1.57052571e-12    
      3      4p_3/2      -3.12630099e+00    -3.12630099e+00    +6.95800623e-12    
      4      5p_3/2      -2.00074429e+00    -2.00074602e+00    +8.63810186e-07    
      5      6p_3/2      -1.38785686e+00    -1.38935143e+00    +1.07689237e-03    
      6      7p_3/2      -9.69506342e-01    -1.02071331e+00    +5.28175648e-02    
      7      8p_3/2      -4.97471413e-01    -7.81461386e-01    +5.70866917e-01    
      8      9p_3/2      +1.08476199e-01    -6.17436217e-01    +6.69190495e+00    
      9     10p_3/2      +8.40255446e-01    -5.00113220e-01    +1.59519188e+00    
     10    

  2p_1/2::  en [a.u.] = -1.3022004e+00;   self-cons'cy = 3.0526e-04  [4.4904e-04 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2959553e+00;   self-cons'cy = 1.0034e-04  [1.5804e-04 for sym-block kappa = -2]

Iteration 11 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3900160e+01;   self-cons'cy = 1.8524e-05  [7.5785e-04 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1159542e+00;   self-cons'cy = 1.1114e-04  [7.5785e-04 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3019061e+00;   self-cons'cy = 1.1300e-04  [1.6798e-04 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2959690e+00;   self-cons'cy = 5.2844e-06  [5.2844e-06 for sym-block kappa = -2]

Iteration 12 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3900534e+01;   self-cons'cy = 5.5146e-06  [2.1346e-04 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1160933e+00;   self-cons'cy = 3.2861e-05  [2.1346e-04 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3019583e+00;   self-cons'cy = 2.0057e-05  [3.0471e-

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_1/2      -1.25208597e+01    -1.25208597e+01    +2.04201324e-10    
      2      3p_1/2      -5.56297036e+00    -5.56297036e+00    +1.64931604e-10    
      3      4p_1/2      -3.12838807e+00    -3.12838807e+00    +1.34509547e-10    
      4      5p_1/2      -2.00181276e+00    -2.00181447e+00    +8.53403101e-07    
      5      6p_1/2      -1.38848496e+00    -1.38996969e+00    +1.06931713e-03    
      6      7p_1/2      -9.70059087e-01    -1.02110261e+00    +5.26189858e-02    
      7      8p_1/2      -4.98191784e-01    -7.81722173e-01    +5.69118958e-01    
      8      9p_1/2      +1.07578533e-01    -6.17619366e-01    +6.74110231e+00    
      9     10p_1/2      +8.39194137e-01    -5.00246729e-01    +1.59610370e+00    
     10    

Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.7585337e+01;   self-cons'cy = 2.8951e-01  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -1.7205500e+00;   self-cons'cy = 7.5837e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.2310526e+00;   self-cons'cy = 8.2096e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.7275423e+00;   self-cons'cy = 7.5723e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3537326e+01;   self-cons'cy = 9.7378e-02  [7.8021e-01 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.3332430e+00;   self-cons'cy = 3.1911e-01  [7.8021e-01 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.2932676e+00;   self-cons'cy = 3.0140e-01  [3.1761e-01 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.9802529e+00;   self-cons'cy = 6.8157e-02  [6.8157e-02 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.1783434e+01;   self-c

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=2.9792e+01,  kappa=-3 [mpt=1116, r[mtp]=8.0980e+00, smallest eigenvalue=3.4537e-13].
>> Pure-sine normalized continuum orbital with normalization constant N=2.7363e+00 and phase phi=1.8584e+00 at r=8.0980e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave d_5/2 ... done. 
>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=2.9792e+01,  kappa=2 [mpt=1116, r[mtp]=8.0980e+00, smallest eigenvalue=-2.8702e-13].
>> Pure-sine normalized continuum orbital with normalization constant N=2.7364e+00 and phase phi=1.8596e+00 at r=8.0980e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave d_3/2 ... done. 
>> (

 
  Auger rates (without angular parameters): 

  ----------------------------------------------------------------------------------------------------------
       i-level-f           i--J^P--f           Energy      Electron energy    Auger rate        alpha_2      
                                                [eV]             [eV]            [1/s]                        
  ----------------------------------------------------------------------------------------------------------
         1 --    1       1/2 + --> 2 +      -2.631778e+03    8.106703e+02    5.476915e+10       0.0000e+00    
         1 --    2       1/2 + --> 1 +      -2.631778e+03    8.105888e+02    1.249266e+08       0.0000e+00    
         1 --    3       1/2 + --> 0 +      -2.631778e+03    8.105505e+02    5.753618e+09       0.0000e+00    
         1 --    4       1/2 + --> 2 +      -2.631778e+03    8.072747e+02    2.391873e+14       0.0000e+00    
         1 --    5       1/2 + --> 0 +      -2.631778e+03    8.041767


This computation just applies simple *Bessel-type* continuum orbitals which are normalized on the basis of a pure sine behaviour at large radii. Moreover, the Auger amplitudes are computed by including the Coulomb interaction. Further control about this computation can be gained by re-defining the method for the integration of the continuum orbitals as well as by means of the `Auger.Settings`. For example, we can use a B-spline-Galerkin method and a normalization for a pure asymptotic Coulomb field to generate all continuum orbitals of interest:


In [3]:
setDefaults("method: continuum, Galerkin")
setDefaults("method: normalization, pure Coulomb")


Moreover, we can first have a look how the standard settings are defined at present: 


In [4]:
? AutoIonization.Settings

`struct  Settings  <:  AbstractProcessSettings`  ... defines a type for the details and parameters of computing Auger lines.

```
+ calcAnisotropy      ::Bool               ... True, if the intrinsic alpha_2,4 angular parameters are to be 
                                               calculated, and false otherwise.
+ printBefore         ::Bool               ... True, if all energies and lines are printed before their evaluation.
+ lineSelection       ::LineSelection      ... Specifies the selected levels, if any.
+ minAugerEnergy      ::Float64            ... Minimum energy of free (Auger) electrons to be included.
+ maxAugerEnergy      ::Float64            ... Maximum energy of free (Auger) electrons to be included.
+ maxKappa            ::Int64              ... Maximum kappa value of partial waves to be included.
+ operator            ::AbstractEeInteraction   
    ... Auger operator that is to be used for evaluating the Auger amplitudes; allowed values are: 
        CoulombInteraction(), BreitInteraction(), ...
```

---

`AutoIonization.Settings()`  ... constructor for the default values of AutoIonization line computations

---

`AutoIonization.Settings(set::AutoIonization.Settings;`

```
    calcAnisotropy=..,      printBefore=..,              
    minAugerEnergy=..,      maxAugerEnergy=..,          maxKappa=..,            operator=..)
                
... constructor for modifying the given AutoIonization.Settings by 'overwriting' the previously selected parameters.
```


In [5]:
defaultSettings = AutoIonization.Settings()

calcAnisotropy:                false  
printBefore:                   false  
lineSelection:                 Inactive LineSelection.  
minAugerEnergy:                0.0  
maxAugerEnergy:                1.0e6  
maxKappa:                      100  
operator:                      CoulombInteraction()  



Obviously, neither the Auger energies nor the symmetries of the (outgoing) partial wave are restricted. Apart from the Coulomb interaction, we could just included the Breit interaction (mainly for test purposes) or both, Coulomb+Breit together, and independent of whether this interaction has been incorporated in the SCF and CI computations before. In addition, we can also calculate the angular anisotropy coefficients as well as specify individual lines or symmetries to be included. We here leave the selection of individual lines to later and define the settings as:


In [6]:
autoSettings = AutoIonization.Settings(defaultSettings, printBefore=true, maxKappa=4)

calcAnisotropy:                false  
printBefore:                   true  
lineSelection:                 Inactive LineSelection.  
minAugerEnergy:                0.0  
maxAugerEnergy:                1.0e6  
maxKappa:                      4  
operator:                      CoulombInteraction()  



and re-run the computations with these settings:


In [7]:
wb = Atomic.Computation(wa, processSettings = autoSettings )
perform(wb)


... in performSCF ...
>> include Configuration: 1s_1/2^1 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
>> (Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 10.0 with mass = 20.5, radius R = 2.8580081360657426 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -5.00667217e+01    -5.00667420e+01    +4.05066052e-07    
      2      2s_1/2      -1.25208571e+01    -1.25208597e+01    +2.03418026e-07    
      3      3s_1/2      -5.56296960e+00    -5.56297036e+00    +1.35691297e-07    
      4      4s_1/2      -3.12838775e+00    -

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_3/2      -1.25041630e+01    -1.25041630e+01    -4.87554157e-13    
      2      3p_3/2      -5.55802266e+00    -5.55802266e+00    -1.57052571e-12    
      3      4p_3/2      -3.12630099e+00    -3.12630099e+00    +6.95800623e-12    
      4      5p_3/2      -2.00074429e+00    -2.00074602e+00    +8.63810186e-07    
      5      6p_3/2      -1.38785686e+00    -1.38935143e+00    +1.07689237e-03    
      6      7p_3/2      -9.69506342e-01    -1.02071331e+00    +5.28175648e-02    
      7      8p_3/2      -4.97471413e-01    -7.81461386e-01    +5.70866917e-01    
      8      9p_3/2      +1.08476199e-01    -6.17436217e-01    +6.69190495e+00    
      9     10p_3/2      +8.40255446e-01    -5.00113220e-01    +1.59519188e+00    
     10    

  1s_1/2::  en [a.u.] = -3.3901416e+01;   self-cons'cy = 3.4772e-05  [1.5076e-03 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1164246e+00;   self-cons'cy = 2.1016e-04  [1.5076e-03 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3022004e+00;   self-cons'cy = 3.0526e-04  [4.4904e-04 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2959553e+00;   self-cons'cy = 1.0034e-04  [1.5804e-04 for sym-block kappa = -2]

Iteration 11 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3900160e+01;   self-cons'cy = 1.8524e-05  [7.5785e-04 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -2.1159542e+00;   self-cons'cy = 1.1114e-04  [7.5785e-04 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.3019061e+00;   self-cons'cy = 1.1300e-04  [1.6798e-04 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.2959690e+00;   self-cons'cy = 5.2844e-06  [5.2844e-06 for sym-block kappa = -2]

Iteration 12 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3900534e+01;   self-cons'cy = 5.5146e-06  [2.1346e-

  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      2p_1/2      -1.25208597e+01    -1.25208597e+01    +2.04201324e-10    
      2      3p_1/2      -5.56297036e+00    -5.56297036e+00    +1.64931604e-10    
      3      4p_1/2      -3.12838807e+00    -3.12838807e+00    +1.34509547e-10    
      4      5p_1/2      -2.00181276e+00    -2.00181447e+00    +8.53403101e-07    
      5      6p_1/2      -1.38848496e+00    -1.38996969e+00    +1.06931713e-03    
      6      7p_1/2      -9.70059087e-01    -1.02110261e+00    +5.26189858e-02    
      7      8p_1/2      -4.98191784e-01    -7.81722173e-01    +5.69118958e-01    
      8      9p_1/2      +1.07578533e-01    -6.17619366e-01    +6.74110231e+00    
      9     10p_1/2      +8.39194137e-01    -5.00246729e-01    +1.59610370e+00    
     10    

Iteration 1 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.7585337e+01;   self-cons'cy = 2.8951e-01  [1.0000e+02 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -1.7205500e+00;   self-cons'cy = 7.5837e-01  [1.0000e+02 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -1.2310526e+00;   self-cons'cy = 8.2096e-01  [1.0000e+02 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.7275423e+00;   self-cons'cy = 7.5723e-01  [1.0000e+02 for sym-block kappa = -2]

Iteration 2 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.3537326e+01;   self-cons'cy = 9.7378e-02  [7.8021e-01 for sym-block kappa = -1]
  2s_1/2::  en [a.u.] = -3.3332430e+00;   self-cons'cy = 3.1911e-01  [7.8021e-01 for sym-block kappa = -1]
  2p_1/2::  en [a.u.] = -2.2932676e+00;   self-cons'cy = 3.0140e-01  [3.1761e-01 for sym-block kappa = 1]
  2p_3/2::  en [a.u.] = -1.9802529e+00;   self-cons'cy = 6.8157e-02  [6.8157e-02 for sym-block kappa = -2]

Iteration 3 for symmetries ... 
  1s_1/2::  en [a.u.] = -3.1783434e+01;   self-c

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=2.9792e+01,  kappa=-3 [mpt=1116, r[mtp]=8.0980e+00, smallest eigenvalue=3.4537e-13].
>> Radial potential with effective charge Zbar=2.0000e+00 (Delta-Zbar=0.0000e+00) at r=1.0062e+01 a.u.
>> (Relativistic) Coulomb-sine normalized continuum orbital with normalization constant N=2.7373e+00 and phase phi=8.9241e-01 at r=8.0980e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave d_5/2 ... done. 
>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=2.9792e+01,  kappa=2 [mpt=1116, r[mtp]=8.0980e+00, smallest eigenvalue=-2.8702e-13].
>> Radial potential with effective charge Zbar=2.0000e+00 (Delta-Zbar=0.0000e+00) at r=1.0062e+01 a.u.
>> (Relativistic) Coulomb-sine normalized continuum orbital with normalization c

>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=2.8372e+01,  kappa=-2 [mpt=1116, r[mtp]=8.0980e+00, smallest eigenvalue=-1.6950e-13].
>> Radial potential with effective charge Zbar=2.0000e+00 (Delta-Zbar=0.0000e+00) at r=1.0062e+01 a.u.
>> (Relativistic) Coulomb-sine normalized continuum orbital with normalization constant N=2.7811e+00 and phase phi=1.2842e+00 at r=8.0980e+00 a.u.
Compute (CoulombInteraction()) Auger matrix of dimension 2 x 1 in the continuum- and initial-state bases for the transition [1- ...] and for partial wave p_3/2 ... done. 
>> (Re-) Define a storage array for various B-spline matrices:
>> Continuum B-spline-Galerkin orbital for energy=2.8372e+01,  kappa=1 [mpt=1116, r[mtp]=8.0980e+00, smallest eigenvalue=-1.5063e-13].
>> Radial potential with effective charge Zbar=2.0000e+00 (Delta-Zbar=0.0000e+00) at r=1.0062e+01 a.u.
>> (Relativistic) Coulomb-sine normalized continuum orbital with normalization 

In the setting above, we could have specified also a computation for just a few selected transitions or just for transitions within a given energy interval. We shall leave the discussion of these *options* to one of the next tutorials.